<a href="https://colab.research.google.com/github/GwiHwan-Go/Detect_SC/blob/main/Collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자살 모니터링을 위한 데이터 수집 코드
### DCinside 우을증 갤러리 데이터 수집

이 코드를 통해 수집한 데이터는 ```Data Analysis.ipynb```에서 자세히 보실 수 있습니다.
### 참고 자료

- [Kokosei J](!https://m.blog.naver.com/ons15/221788312880)
- [scrapehero](!https://www.scrapehero.com/)
- [proxy 사용](!how-to-rotate-proxies-and-ip-addresses-using-python-3/)

In [3]:
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd 
import requests
from lxml.html import fromstring
import random


In [2]:
###PARAMS
headers = {"User-Agent": "Monika (Android;)", #User-Agent를 줘서, 사이트의 차단을 피합니다.
           "Cookie": "list_count=100"}
gallid = 'depression_new1' #DC inside의 우울증 갤러리의 website 주소입니다.

# 포맷으로 URL 생성
url = "https://m.dcinside.com/board/{}?page={{}}".format(gallid)
url_desktop = "https://gall.dcinside.com/{{}}board/lists/?id={}&list_num=100&page=1".format(gallid)
url_fortxt = "https://gall.dcinside.com/{{}}board/view/?id={}&no=".format(gallid)
proxy_pool={} # 사이트의 차단을 피하기위한 proxy 집합입니다.

def get_proxies(update=False) :

  # 본 함수는 'https://free-proxy-list.net/' 에서 proxy 를 수집해오는 코드입니다.
  # 만약 위 사이트의 구조가 변경되었다면, 제대로 실행이 되지 않을 수도 있습니다.

  global proxy_pool

  url = 'https://free-proxy-list.net/'
  response = requests.get(url)
  parser = fromstring(response.text)

  if update :
    proxies = update
  else :
    proxies = set()
  for i in parser.xpath('//tbody/tr')[:30]:

    if i.xpath('.//td[7][contains(text(),"yes")]'):
    #Grabbing IP and corresponding PORT
      proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
      proxies.add(proxy)
    
  return proxies

def delete_proxy(li, ele) :
  ## 수집한 proxy 중 인자로 받은 ele 를 지웁니다.
  li.remove(ele)
  return li

def get_end_page_of_site() :

    global url_fortxt
     # 마지막 페이지 번호 파싱하기 위해 데스크탑 페이지 접속
    r = requests.get(url_desktop.format(""), headers = {"User-Agent": "Monika"})
    if "location.replace" in r.text:
        # 마갤이므로 마갤주소로 변환 후 다시 리퀘스트 보냄
        r = requests.get(url_desktop.format("mgallery/"), headers = {"User-Agent": "Monika"})
        # txt파일에 들어갈 url 형식 변환
        url_fortxt = url_fortxt.format("mgallery/")
    else:
        url_fortxt = url_fortxt.format("")
    # 갤 이름, 마지막 페이지 번호 파싱
    soup = bs(r.text, "html.parser")
    # gallname = soup.find("meta", {"name":"title"})['content']
    endpage = int(soup.find("a", {"class":"page_end"})['href'].split("=")[-2].split("&")[0])

    return endpage

def get_page(pagenumb, start, end):

    global proxy_pool, url

    proxy_pool = get_proxies(proxy_pool)
    proxy = random.sample(proxy_pool,1)[0]
    print(pagenumb, "번 페이지 파싱 시작")

    try :

      r = requests.get(url.format(pagenumb), 
                        headers = headers,
                        proxies={"http": proxy}
                        )
      soup = bs(r.text, "html.parser")

      rtnlist = []

      for x in soup.find('ul', {'class': "gall-detail-lst"}).find_all("div", {"class":"gall-detail-lnktb"}):
          # 글제목 추출
          title = x.find('span', {"class":"subjectin"}).text

          # 링크, 글번호 추출
          link = x.find('a')['href']
          id = int(link.split("/")[-1].split("?")[0])

          # 데이터 추출 시 기존 자료와 중복 장지
          if id > start and id < end : 
            print(f"cur_id : {id}, is in the range of {start}~{end}")
            break

          # 작성자 추출
          author = x.find_all("li")[0].text
          date = x.find_all("li")[1].text
          if ":" in date :
            date = f"{localtime.tm_mon}.{localtime.tm_mday}"
          # print(author + " - " + title + ", " + id + " 번 게시글 확인")
          
          rtnlist.append([id, title, author, date, link])
          # print(rtnlist)
      # print(f"Collected {len(rtnlist)} data")

      return rtnlist

    except :
      # 연결이 안될 시 다른 proxy 로 재 시도합니다.
      print("Skipping. Conection errors, try again with another proxy")
      if len(proxy_pool) == 0 :
        print("No available proxy, updating...")
        time.sleep(10)
        proxy_pool = get_proxies(proxy_pool)
      else :
        print("Remove this proxy and try it with another proxy")
        proxy_pool.remove(proxy)
        return []

## 데이터 수집 시작

본격적으로 데이터 수집을 위한 준비를 마쳤습니다.

아래에서 데이터 수집을 시작합니다.

colab의 특성상 비정상적으로 종료될 경우를 대비해서, 

except 부분에서 현재까지 실행한 파일을 저장하는 코드를 구현하였습니다.

In [ ]:
#MAIN

results = []
first_id,last_id = 0, 0 # 기존 수집한 데이터가 있을 때 중복을 방지하기 위해 존재합니다.
localtime = time.localtime()
starttime = time.time()
endpage = 10 # 테스트용 # get_end_page_of_site() 본 코드
print("마지막 페이지:", endpage)

print("5 초 후 파싱작업을 시작합니다...")
# time.sleep(5)
try :

  for page in range(100) :
  # for page in range(1,3):
      new_data = get_page(page,first_id,last_id)
      if len(new_data)!=0 :
        results+=new_data
        print(f"Collected {len(results)} data")
  print("파싱 작업 종료. 2 초 후 리스트를 정렬합니다...")
  time.sleep(2)

  print("정렬시작...")
  postlist_sorted = sorted(results, key = lambda x:  int(x[0]))

  # csv로 저장
  df = pd.DataFrame.from_records(postlist_sorted, columns=['id', 'title', 'author', 'link'])
  file_path = save_dir+"/"+f'{localtime.tm_mon}{localtime.tm_mday}{localtime.tm_hour}{localtime.tm_min}.csv'
  df.to_csv(file_path)

  print(f"파일 {file_path}에 저장 완료")
  print("파싱이 완료되었습니다.\n 소요시간: {}초\n프로그램을 종료합니다.".format(time.time() - starttime))

except :

  # 에러났을 때 지금까지 한 거라도 csv로 저장
  postlist_sorted = sorted(results, key = lambda x:  int(x[0]))
  df = pd.DataFrame.from_records(postlist_sorted, columns=['id', 'title', 'author','date', 'link'])
  file_path = save_dir+"/"+f'{localtime.tm_mon}{localtime.tm_mday}{localtime.tm_hour}{localtime.tm_min}.csv'
  df.to_csv(file_path)
  print(f"파일 {file_path}에 저장 완료")
  print("알 지 못할 에러로 프로그램을 종료합니다. \n 소요시간: {}초\n프로그램을 종료합니다.".format(time.time() - starttime))

마지막 페이지: 24874
5 초 후 파싱작업을 시작합니다...
0 번 페이지 파싱 시작
Collected 99 data
1 번 페이지 파싱 시작
Collected 198 data
2 번 페이지 파싱 시작
Collected 298 data
3 번 페이지 파싱 시작
Collected 398 data
4 번 페이지 파싱 시작
Collected 498 data
5 번 페이지 파싱 시작
Collected 598 data
6 번 페이지 파싱 시작
Collected 698 data
7 번 페이지 파싱 시작
Collected 798 data
8 번 페이지 파싱 시작
Collected 898 data
9 번 페이지 파싱 시작
Collected 998 data
10 번 페이지 파싱 시작
Collected 1098 data
11 번 페이지 파싱 시작
Collected 1198 data
12 번 페이지 파싱 시작
Collected 1298 data
13 번 페이지 파싱 시작
Collected 1398 data
14 번 페이지 파싱 시작
Collected 1498 data
15 번 페이지 파싱 시작
Collected 1598 data
16 번 페이지 파싱 시작
Collected 1698 data
17 번 페이지 파싱 시작
Collected 1798 data
18 번 페이지 파싱 시작
Collected 1898 data
19 번 페이지 파싱 시작
Collected 1998 data
20 번 페이지 파싱 시작
Collected 2098 data
21 번 페이지 파싱 시작
Collected 2198 data
22 번 페이지 파싱 시작
Collected 2298 data
23 번 페이지 파싱 시작
Collected 2398 data
24 번 페이지 파싱 시작
Collected 2498 data
25 번 페이지 파싱 시작
Collected 2598 data
26 번 페이지 파싱 시작
Collected 2698 data
27 번 페이지 파싱 시작
Collected 2798 data
28 번 

### 수집한 데이터 확인

왼쪽의 메뉴에서 폴더같이 생긴 아이콘을 클릭해 줍니다.

현재 디렉토리에 새로운 csv 파일이 생겼을 겁니다. 이제 그 파일의 경로를 복사해줍니다.

대개 ```/content/{file_name}/``` 이 경로입니다.

In [4]:
new_data_path = "/content/10302344.csv"
new_data = pd.read_csv(new_data_path)
new_data.head()

,Unnamed: 0,id,title,author,date,link
0,0,4467488,시발 과제쳐하고 왔더니 놀사람들 다자러갔네,김새솔,10.3,https://m.dcinside.com/board/depression_new1/4...
1,1,4467489,하토애미뒤진년은 365일 싸우네 ㄹㅇ,매꿍(223.39),10.3,https://m.dcinside.com/board/depression_new1/4...
2,2,4467490,아니 누구지 코스하는 애들 역겨워서 언급도 안하는데(시비아니고 진심),하토(121.187),10.3,https://m.dcinside.com/board/depression_new1/4...
3,3,4467491,키 모자란다고 놀리고 등급 모자란다고 놀려서 싸우는거임?,뼈삭음,10.3,https://m.dcinside.com/board/depression_new1/4...
4,4,4467492,ㅈ에고무줄묶어서괴사시키고싶은남갤러list.,ㅇㅇ(118.235),10.3,https://m.dcinside.com/board/depression_new1/4...


## 자료 수집 끝!